In [ ]:
pip install stix2

In [ ]:
pip install taxii2-client

In [1]:
from taxii2client.v20 import Server # only specify v20 if your installed version is >= 2.0.0

In [2]:
# Instantiate server and get API Root
server = Server("https://cti-taxii.mitre.org/taxii/")
api_root = server.api_roots[0]

# Print name and ID of all ATT&CK domains available as collections from the TAXII server
for collection in api_root.collections:
    print(collection.title.ljust(20) + collection.id)


Enterprise ATT&CK   95ecc380-afe9-11e4-9b6c-751b66dd541e
PRE-ATT&CK          062767bd-02d2-4b72-84ba-56caef0f8658
Mobile ATT&CK       2f669986-b40b-4423-b720-4396ca6a462b
ICS ATT&CK          02c3ef24-9cd4-48f3-a99f-b74ce24f1d34


In [46]:
from stix2 import TAXIICollectionSource, Filter
from taxii2client.v21 import Collection, as_pages # only specify v20 if your installed version is >= 2.0.0

In [3]:
collections = {
    "enterprise_attack": "95ecc380-afe9-11e4-9b6c-751b66dd541e",
    "mobile_attack": "2f669986-b40b-4423-b720-4396ca6a462b",
    "ics-attack": "02c3ef24-9cd4-48f3-a99f-b74ce24f1d34"
}

In [6]:
# Establish TAXII2 Collection instance for Enterprise ATT&CK 
collection = Collection(f"https://cti-taxii.mitre.org/stix/collections/{collections['enterprise_attack']}/")

# Supply the collection to TAXIICollection
src = TAXIICollectionSource(collection)

In [91]:
# Helper function to filter revoked and depcreated objects (need this bc filter behaves differently for TAXII server)
#https://github.com/mitre/cti/blob/master/USAGE.md#Working-with-deprecated-and-revoked-objects

def remove_revoked_deprecated(stix_objects):
    """Remove any revoked or deprecated objects from queries made to the data source"""
    # Note we use .get() because the property may not be present in the JSON data. The default is False
    # if the property is not set.
    return list(
        filter(
            lambda x: x.get("x_mitre_deprecated", False) is False and x.get("revoked", False) is False,
            stix_objects
        )
    )

In [91]:
techniques = src.query([
    Filter("type", "=", "attack-pattern")
])

techniques = remove_revoked_deprecated(techniques)

len(techniques)

[taxii2client.v20] [WARNING ] [2023-02-21 16:52:55,158] TAXII Server Response did not include 'Content-Range' header - results could be incomplete.
[taxii2client.v20] [WARNING ] [2023-02-21 16:52:55,184] TAXII Server Response with different amount of objects! Setting per_request=737


594

In [131]:
print(dict(techniques[0]).keys())


dict_keys(['type', 'id', 'created_by_ref', 'created', 'modified', 'name', 'description', 'kill_chain_phases', 'revoked', 'external_references', 'object_marking_refs', 'x_mitre_attack_spec_version', 'x_mitre_contributors', 'x_mitre_data_sources', 'x_mitre_deprecated', 'x_mitre_detection', 'x_mitre_domains', 'x_mitre_is_subtechnique', 'x_mitre_modified_by_ref', 'x_mitre_platforms', 'x_mitre_version'])


In [145]:
dict(techniques[0])

{'type': 'attack-pattern',
 'id': 'attack-pattern--005cc321-08ce-4d17-b1ea-cb5275926520',
 'created_by_ref': 'identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5',
 'created': '2022-09-30T21:18:41.930Z',
 'modified': '2022-10-20T19:56:18.579Z',
 'name': 'Socket Filters',
 'description': 'Adversaries may attach filters to a network socket to monitor then activate backdoors used for persistence or command and control. With elevated permissions, adversaries can use features such as the `libpcap` library to open sockets and install filters to allow or disallow certain types of data to come through the socket. The filter may apply to all traffic passing through the specified network interface (or every interface if not specified). When the network interface receives a packet matching the filter criteria, additional actions can be triggered on the host, such as activation of a reverse shell.\n\nTo establish a connection, an adversary sends a crafted packet to the targeted host that matches the in

In [148]:
# Initialize list to hold Enterprise ATT&CK content
attack_list = []

for i in range(len(techniques)):
    #initialize dictionary
    attack_dict = {}
    
    #extract technique ID
    t_id = dict(techniques[i].external_references[0]).get('external_id')
    attack_dict['technique'] = t_id
    
    #extract technique name
    attack_dict['name'] = techniques[i].get('name')
    
    #extract data sources
    attack_dict['data_sources'] = techniques[i].get('x_mitre_data_sources')
    
    #append technique dictionary to list
    attack_list.append(attack_dict)
    
    


len(attack_list)        
# print(json.dumps(attack_list, indent=4))

594

In [163]:
# Get all data sources in Enterprise ATT&CK
all_data_srcs = []

for tech in techniques:
    if 'x_mitre_data_sources' in tech:
        all_data_srcs += [
            data_src for data_src in tech.x_mitre_data_sources
            if data_src not in all_data_srcs
        ]


all_data_srcs.sort()
all_data_srcs


['Active Directory: Active Directory Credential Request',
 'Active Directory: Active Directory Object Access',
 'Active Directory: Active Directory Object Creation',
 'Active Directory: Active Directory Object Deletion',
 'Active Directory: Active Directory Object Modification',
 'Application Log: Application Log Content',
 'Certificate: Certificate Registration',
 'Cloud Service: Cloud Service Disable',
 'Cloud Service: Cloud Service Enumeration',
 'Cloud Service: Cloud Service Modification',
 'Cloud Storage: Cloud Storage Access',
 'Cloud Storage: Cloud Storage Creation',
 'Cloud Storage: Cloud Storage Deletion',
 'Cloud Storage: Cloud Storage Enumeration',
 'Cloud Storage: Cloud Storage Metadata',
 'Cloud Storage: Cloud Storage Modification',
 'Command: Command Execution',
 'Container: Container Creation',
 'Container: Container Enumeration',
 'Container: Container Start',
 'Domain Name: Active DNS',
 'Domain Name: Domain Registration',
 'Domain Name: Passive DNS',
 'Drive: Drive Ac

In [56]:
# Initialize dictionary to hold Enterprise ATT&CK content
attack = {}

# Create filters to retrieve content from Enterprise ATT&CK
filter_objs = {"techniques": Filter("type", "=", "attack-pattern"),
    "mitigations": Filter("type", "=", "course-of-action"),
    "groups": Filter("type", "=", "intrusion-set"),
    "malware": Filter("type", "=", "malware"),
    "tools": Filter("type", "=", "tool"),
    "relationships": Filter("type", "=", "relationship")
}

In [75]:
# Retrieve all Enterprise ATT&CK content
for key in filter_objs:
    attack[key] = src.query(filter_objs[key])

# For visual purposes, print the first technique received
# print(attack["techniques"][0].serialize(pretty=True))
len(attack)

[taxii2client.v20] [WARNING ] [2023-02-21 16:22:44,024] TAXII Server Response did not include 'Content-Range' header - results could be incomplete.
[taxii2client.v20] [WARNING ] [2023-02-21 16:22:44,045] TAXII Server Response with different amount of objects! Setting per_request=737
[taxii2client.v20] [WARNING ] [2023-02-21 16:22:46,837] TAXII Server Response did not include 'Content-Range' header - results could be incomplete.
[taxii2client.v20] [WARNING ] [2023-02-21 16:22:46,841] TAXII Server Response with different amount of objects! Setting per_request=267
[taxii2client.v20] [WARNING ] [2023-02-21 16:22:49,780] TAXII Server Response did not include 'Content-Range' header - results could be incomplete.
[taxii2client.v20] [WARNING ] [2023-02-21 16:22:49,785] TAXII Server Response with different amount of objects! Setting per_request=147
[taxii2client.v20] [WARNING ] [2023-02-21 16:22:53,250] TAXII Server Response did not include 'Content-Range' header - results could be incomplete.


6

In [ ]:
t1134 = src.query([ 
    Filter("external_references.external_id", "=", "T1134"), 
    Filter("type", "=", "attack-pattern")])[0]


In [79]:
src.query([ 
    Filter("external_references.external_id", "=", "T1134")])

[taxii2client.v20] [WARNING ] [2023-02-21 16:26:39,035] TAXII Server Response did not include 'Content-Range' header - results could be incomplete.
[taxii2client.v20] [WARNING ] [2023-02-21 16:26:41,088] TAXII Server Response did not include 'Content-Range' header - results could be incomplete.
[taxii2client.v20] [WARNING ] [2023-02-21 16:26:44,468] TAXII Server Response did not include 'Content-Range' header - results could be incomplete.
[taxii2client.v20] [WARNING ] [2023-02-21 16:26:46,286] TAXII Server Response did not include 'Content-Range' header - results could be incomplete.


[CourseOfAction(type='course-of-action', id='course-of-action--c61fee9f-16fb-4f8c-bbf0-869093fcd4a6', created_by_ref='identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5', created='2018-10-17T00:14:20.652Z', modified='2019-07-24T14:29:27.367Z', name='Access Token Manipulation Mitigation', description='Access tokens are an integral part of the security system within Windows and cannot be turned off. However, an attacker must already have administrator level access on the local system to make full use of this technique; be sure to restrict users and accounts to the least privileges they require to do their job.\n\nAny user can also spoof access tokens if they have legitimate credentials. Follow mitigation guidelines for preventing adversary use of [Valid Accounts](https://attack.mitre.org/techniques/T1078). Limit permissions so that users and user groups cannot create tokens. This setting should be defined for the local system account only. GPO: Computer Configuration > [Policies] > Windows S

In [64]:
print(t1134.serialize(pretty=True))

{
    "type": "attack-pattern",
    "id": "attack-pattern--dcaa092b-7de9-4a21-977f-7fcb77e89c48",
    "created_by_ref": "identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5",
    "created": "2017-12-14T16:46:06.044Z",
    "modified": "2022-05-03T02:14:43.557Z",
    "name": "Access Token Manipulation",
    "description": "Adversaries may modify access tokens to operate under a different user or system security context to perform actions and bypass access controls. Windows uses access tokens to determine the ownership of a running process. A user can manipulate access tokens to make a running process appear as though it is the child of a different process or belongs to someone other than the user that started the process. When this occurs, the process also takes on the security context associated with the new token.\n\nAn adversary can use built-in Windows API functions to copy access tokens from existing processes; this is known as token stealing. These token can then be applied to an existi

In [43]:
t1134.get('x_mitre_data_sources')

['Process: Process Metadata',
 'Process: OS API Execution',
 'User Account: User Account Metadata',
 'Process: Process Creation',
 'Command: Command Execution',
 'Active Directory: Active Directory Object Modification']